In [15]:
import torch
import torch.nn as nn
from torchvision import transforms

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}:", torch.cuda.get_device_name())

Using cuda: NVIDIA GeForce RTX 3050 Laptop GPU


### Hyperparameters

In [9]:
input_size = 784 # 28x28 images flattened
hidden_size = 128
num_classes = 10 # 10 digits
num_epochs = 5
batch_size = 100
learning_rate = 0.001


**input_size:**  
Each MNIST image is 28×28 pixels, which we flatten to a 784-element vector  

**hidden_size:**  
Number of neurons in our hidden layer  

**num_classes:**  
We have 10 possible outputs (digits 0-9)  

**num_epochs:**  
Number of complete passes through the training dataset  

**batch_size:**  
Number of images processed together  

**learning_rate:**  
Controls how much we adjust our model in response to errors  

### Data Preparation

In [16]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])